https://docs.python.org/3/reference/import.html

https://docs.python.org/3/library/importlib.html

In [1]:
import sys
import importlib
import os.path
from IPython import get_ipython
import nbformat
import io
import time

NBVERSION = 4

In [2]:
def compile_to_py(nb_path,py_path):

    shell = get_ipython()

    # load the notebook object
    with io.open(nb_path, 'r', encoding='utf-8') as f:
        nb = nbformat.read(f,NBVERSION)

    with io.open(py_path,'w',encoding='utf-8') as pyf:
        pyf.write(u'## Compiled from {} on {}\n'.format(nb_path,time.ctime()))
        for cell in nb['cells']:
            if cell['cell_type'] == 'code':
                # transform the input to executable Python
                ##print ("Source",cell['source'])
                ec = cell['execution_count']
                code = shell.input_transformer_manager.transform_cell(cell['source'])
                if code.startswith('##test:'):
                    continue
                if code.startswith('get_ipython().run_cell_magic('):
                    continue
                if code.startswith('## Test Section:'):
                    pyf.write(u'\n## Import ended by "## Test Section:"\n')
                    break
                if code.startswith('#### End Import ####'):
                    pyf.write(u'\n## Import ended by "#### End Import ####"\n')
                    break

                pyf.write(u'\n')
                pyf.write(u'## In [{}]:\n'.format(' ' if ec is None else ec))
                pyf.write(code)


In [3]:
def must_compile(nb_path,py_path):
    if not os.path.exists(py_path):
        return True
    nbt = os.path.getmtime(nb_path)
    pyt = os.path.getmtime(py_path)
    return pyt < nbt

In [48]:
class NBFinder(importlib.abc.PathEntryFinder):
    
    DEBUG = False
    
    def find_spec(self,fullname,path,target=None):
        if self.DEBUG:
            print('find_spec:',fullname,path,target)
        if path is None:
            path = sys.path
        modname = fullname.split('.')[-1]
        filename = modname + '.ipynb'
        for p in path:
            fullpath = os.path.join(p,filename)
            if os.path.isfile(fullpath):
                if self.DEBUG:
                    print('  found:',fullpath)
                pypath = fullpath[:-5] + 'py'
                if must_compile(fullpath,pypath):
                    if self.DEBUG:
                        print('  compiling to:',pypath)
                    compile_to_py(fullpath,pypath)
                loader = importlib.machinery.SourceFileLoader(fullname,pypath)
                spec = importlib.machinery.ModuleSpec(fullname,loader,origin=pypath)
                return spec
        
        return None
    
    pass
__TheNBFinder = NBFinder()

In [49]:
if __TheNBFinder not in sys.meta_path:
    sys.meta_path[:1] = [__TheNBFinder]
sys.meta_path

 _frozen_importlib.FrozenImporter,
 _frozen_importlib_external.PathFinder,

In [50]:
mn = 'Frame2D.testnb'
if mn in sys.modules:
    del sys.modules[mn]
    print(mn,'deleted')

Frame2D.testnb deleted


In [51]:
import Frame2D.testnb as tt

In [52]:
tt.T

8421

In [8]:
from Frame2D import Frame2D

In [9]:
Frame2D

(True, False)

In [10]:
from Frame2D import testnb

In [11]:
testnb.T

123

In [12]:
import othertest

find_spec: othertest None None


In [13]:
othertest.T

456

In [24]:
p = ['/files/home/nholtz/work/git/structural-analysis/matrix-methods/frame2d/Frame2D']
fn = 'Frame2D.test'
for finder in sys.meta_path:
    if hasattr(finder,'find_spec'):
        s = finder.find_spec(fn,p)
        print(finder,s)

find_spec: Frame2D.test ['/files/home/nholtz/work/git/structural-analysis/matrix-methods/frame2d/Frame2D'] None
<__main__.NBFinder object at 0x7f6962ce2438> None
<class '_frozen_importlib.FrozenImporter'> None
<class '_frozen_importlib_external.PathFinder'> ModuleSpec(name='Frame2D.test', loader=<_frozen_importlib_external.SourceFileLoader object at 0x7f6970567550>, origin='/files/home/nholtz/work/git/structural-analysis/matrix-methods/frame2d/Frame2D/test.py')


In [28]:
s.loader.path

'/files/home/nholtz/work/git/structural-analysis/matrix-methods/frame2d/Frame2D/test.py'